# Geospatial Data & Mapping In Python

**Step 1**

In [ ]:
# Install geopandas and descartes libraries
!conda install --yes geopandas descartes contextily

In [ ]:
# Install Folium Library
!pip install contextily folium

# If the above didn't work, try this
#!conda install --yes folium -c conda-forge

In [ ]:
#for pandas/geopandas
import pandas as pd
import geopandas as gpd
import descartes
import contextily
import matplotlib.pyplot as plt

# for folium
import folium

## Set Up Your Filepath
I am doing this here to make the rest of the workshop smoother. We will be importing files in order to work with them. It will make sense why I am doing this later in the workshop

**Step 2**

In [ ]:
# example: /Users/ep9k/Desktop/

my_path = './cville_gis_data'

## Pandas - A quick overview

If you are not familiar with it, [Pandas](https://pandas.pydata.org/) is a widely used python library (along with its counterparts Matplotlib, NumPy, SciPy, etc..) which allows data analysis and manipulation. Pandas uses **dataframes**, which are a spreadsheet-like data type, to allow you to manipulate your data much in the same way you can in Microsoft Excel or similar programs.

**Step 3**

In [ ]:
# Create an example dataframe

# initialize data of lists.
data = {'Name': ['Amy', 'Nick', 'John', 'Emily'],
        'Age': [20, 21, 19, 18]}
  
# Create DataFrame
df = pd.DataFrame(data)
  
# Print the output.
df

## GeoPandas

**From the [GeoPandas Documentation](https://geopandas.org/en/stable/index.html)**: GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by [Shapely](https://shapely.readthedocs.io/en/stable/). Geopandas further depends on [Fiona](https://fiona.readthedocs.io/en/latest/) for file access and [MatPlotLib](https://matplotlib.org/) for plotting.

Basically, GeoPandas allows python users to do mapping, plotting, and geospatial operations all within python. No more mixing between other programs!

#### A Note on our data source

All the data we are using today is from the [City of Charlottesville's Open Data Portal](https://opendata.charlottesville.org/). This is freely available data that the city provides to the public. 

**Step 4**

In [ ]:
# Reading GeoJSON.  GeoJSON is a common geospatial data format that you will encounter. It is basically just structured text.
cville_boundary = gpd.read_file(my_path + '/Charlottesville_Basemap.geojson')

In [ ]:
cville_boundary.plot()

### GeoPandas GeoDataFrame

In GeoPandas, the GeoDataFrame is a common data type. This is basically just like a Pandas dataframe but with a geometry component. 

**Step 5**

In [ ]:
print(type(df))                  # normal pandas dataframe
print(type(cville_boundary))     # geopandas geodataframe

In [ ]:
# let's take a look and see what is inside this GeoDataFrame
cville_boundary

In a GeoDataFrame, there is a **geometry** column. This column holds the spatial information about the object(s) in the GeoDataFrame. This will look different depending on what type of object the GeoDataFrame is (point, line, polygon, multipolygon, etc.)

In [ ]:
print(cville_boundary.columns.to_list())
print()
print(cville_boundary['geometry'])

**Step 6**

In [ ]:
# now we read in roads layer. Notice that this is a little more exciting as there are quite a few roads within city limits

roads = gpd.read_file(my_path + '/Road_centerlines.shp')
roads

In [ ]:
# Now let's look at the roads layer.

roads.plot()

### Overlay multiple layers

Plot differenty layers on top of each other, basically stacking them on top of one another.
We are not just limited to two layers, you can stack many layers

**Step 7**

In [ ]:
# why is this setup used for multiple layers??
# because the docs say so! ex: https://geopandas.org/en/stable/docs/user_guide/mapping.html?highlight=multiple%20layers#control-the-order-of-multiple-layers-in-a-plot

fig, ax = plt.subplots(figsize=(10,8))
cville_boundary.plot(ax=ax, edgecolor='black')
roads.plot(ax=ax, color='white')

### Add a basemap

Basemaps are commonly satellite or similar imagery that is used as a locator or for backdrop.
We will use the Contextily library as a basemap in this case

**Step 8**

In [ ]:
# find this code in geopanda docs: https://geopandas.org/en/stable/gallery/plotting_basemap_background.html?highlight=basemap

ax = cville_boundary.plot(figsize=(10,8), alpha=0.5)
contextily.add_basemap(ax, crs=roads.crs)

## Working with Coordinate Reference Systems and Map Projections

This is a big topic that we will not go too far into. Basically, the age old issue that geographers have faced it how to represent a round object (the earth) on a flat image such as a book, paper, computer screen, etc. At some point, coordinate reference systems and map projections were created in order to do that. 

The very short explanation of these two terms...

**Coordinate Reference System (CRS)**
Everything on the earth has a location. Today, this is commonly represented in latitude/longitude or XY coordinates. For example, Charlottesville is located at (38.03, -78.48) or 38.03N, 78.48W. 

**Map Projections**
Though everything on the earth has an exact location, it is not always represented the same. This is where map projections come into play. Basically, a map projection is focused on representing various parts of the earth as accurately as possible. There are hundreds (maybe thousands?) of map projections and they can range in focus from parts of individual states, north america, eastern hemisphere, southern hemisphere, etc. 

**Step 9**

What are we currently using?

In [ ]:
# WGS 1984 is a commonly used map projection that represents the entire world and is measured in decimal degrees
cville_boundary.crs

#### Change the CRS of a layer(s)

If you don't like the CRS your data comes with or need to change it, you can do that

**Step 10**

In [ ]:
#I will change both layers to a CRS that measures in feet
# EPSG 32046 is a Virginia State Plane CRS measured in feet

cville_boundary = cville_boundary.to_crs(epsg = 32046)
roads = roads.to_crs(epsg = 32046)

In [ ]:
# and plot them again. Notice the units of measurement are now different! (Now in feet)

fig, ax = plt.subplots(figsize=(10,8))
cville_boundary.rotate(90)
cville_boundary.plot(ax=ax, edgecolor='black')
roads.plot(ax=ax, color='orange')

## Geospatial Operations

Some of the many amazing things that GeoPandas can do are geospatial operations previously unavailable (to my knowledge) in python. Now, you don't have to do them using another GIS software such as ArcGIS or QGIS or a Geospatial Database such as PostgreSQL. You can do it right in python!

Let's do a few common geospatial operations such as intersections, measuring distances, manipulating the attribute table, etc.

**Step 11**

In [ ]:
neighborhoods = gpd.read_file(my_path + '/neighborhoods.geojson')

neighborhoods.plot(column='NAME', cmap='jet')

#neighborhoods

In [ ]:
# you can symbolize any of the fields in this geodataframe easily!
# One of the other columns in this dataset is 'Planner'. This is not really that meaningful but this is the City of Charlottesville planner who is responsible for this neighborhoo
# Another example of this is the 'Engineer' column

neighborhoods.plot(column='PLANNER')

neighborhoods.plot(column='ENGINEER')

In [ ]:
# convert our new layers to the same CRS as the others
neighborhoods = neighborhoods.to_crs(epsg = 32046)

### Intersections

We will now how layers intersect with each other. First I will make a new layer that is just the Woolen Mills neighborhood

**Step 12**

In [ ]:
# using exactly the same methods as in regular pandas to extract the data I want
woolen_mills = neighborhoods.loc[neighborhoods['NAME'] == 'Woolen Mills']
woolen_mills = woolen_mills.to_crs(epsg = 32046)
woolen_mills.plot()



# plot the new woolen mills layer over the cville_boundary layer
fig, ax = plt.subplots(figsize=(10,8))
cville_boundary.plot(ax=ax, edgecolor='black')
woolen_mills.plot(ax=ax, color='red', edgecolor='black')



In [ ]:
woolen_mills_roads = gpd.overlay(roads, woolen_mills, how='intersection')

#plot all of charlottesville, woolen mills, roads, and the roads in woolen mills
fig, ax = plt.subplots(figsize=(10,8))
cville_boundary.plot(ax=ax, edgecolor='black')
roads.plot(ax=ax, color='white')
woolen_mills.plot(ax=ax, color='red', edgecolor='black')
woolen_mills_roads.plot(ax=ax, color='black')

### Buffers

Another common operation is to create a buffer around an object

**Step 13**

In [ ]:
# first, let's import a few new layers

cemeteries = gpd.read_file(my_path + '/cemeteries.geojson')
cemeteries = cemeteries.to_crs(32046)

# the cemeteries are polygons (shapes) of the entire cemetery. I want to measure distance point to point in this case
# I will create a centroid point for each of the cemeteries
cemeteries['centroid'] = cemeteries.centroid

cemeteries['buffer'] = cemeteries['centroid'].buffer(1320)    #remember, this is being measured in feet. 1320 feet is 1/4 mile
 
cemeteries_buffer = cemeteries['buffer']

fig, ax = plt.subplots(figsize=(10,8))
cemeteries_buffer.plot(ax=ax, color='red', alpha=0.5)
cemeteries.plot(ax=ax, edgecolor='black')
contextily.add_basemap(ax, crs=cemeteries.crs)


### Exporting files

You can export a GeoDataFrame to one of many geospatial file types such as geopackage, ESRI Shapefile, geoJSON, etc.

In [ ]:
#look in your data folder and you should have a new layer with the name 'NEW_GEOPACKAGE'
roads.to_file(my_path + 'NEW_GEOPACKAGE.gpkg', layer='MY_NEW_LAYER', driver='GPKG')


## Folium

[Folium](http://python-visualization.github.io/folium/) is another python mapping library built on the popular Leaflet javascript library. Now, we can make nice web maps with python instead of javascript! The strength of folium is in its mapping capabilities. This is not a data manipulation library in the way that GeoPandas is. 

The following code examples I have taken mostly from the [Folium Docs](http://python-visualization.github.io/folium/quickstart.html) with very slight modifications. 

Also, I will be using publicly available data from the [City of Richmond's GeoHub](https://richmond-geo-hub-cor.hub.arcgis.com/)

**Step 14**

In [ ]:
m = folium.Map(location=[37.5407, -77.436], zoom_start=10)
m

### Add Data to the map

We can add all the same kinds of data we had previously used.

**Step 15**

In [ ]:
# notice that I am streaming this data from a URL provided by the city of Richmond's data portal: https://richmond-geo-hub-cor.hub.arcgis.com/datasets/cor::public-libraries/about
richmond_libraries = f"https://services1.arcgis.com/k3vhq11XkBNeeOfM/arcgis/rest/services/PublicLibrary/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

m = folium.Map(
    location=[37.5407, -77.436],
    zoom_start=11,
)

folium.GeoJson(richmond_libraries, name="geojson").add_to(m)

m

## Choropleth Maps

Folium has some really nice looking mapping capabilities. These go far and beyond the simple plots of geopandas and matplotlib

**Step 16**

In [ ]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

### Using GeoPandas with Folium

We can plot using GeoPandas GeoDataFrames and the like

**Step 17**

In [ ]:
#geopandas docs about plotting with folium: https://geopandas.org/en/stable/gallery/plotting_with_folium.html

volcanoes = pd.read_csv(my_path + '/volcano_data_2010.csv')

# keep only relevant columns
volcanoes = volcanoes.loc[:, ("Year", "Name", "Country", "Latitude", "Longitude", "Type")]
volcanoes

In [ ]:
# create point geometries for the volcanoes
geometry = gpd.points_from_xy(volcanoes['Longitude'], volcanoes['Latitude'])    # this makes point objects in an array

volcanoes['geometry'] = geometry     # add points as new column to geodataframe
volcanoes

In [ ]:
#make map with OpenStreetMap basemap

# OpenStreetMap
mymap = folium.Map(location = [0, 0], tiles='OpenStreetMap' , zoom_start = 1)
mymap

### Make Markers for Each Volcano in Folium Map

This is tricky stuff but basically we are iterating over each point in the geodataframe and creating a marker for it

In [ ]:
# Create Markers for Volcanoes

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in volcanoes.geometry ]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    #assign a color marker for the type of volcano, Strato being the most common
    if volcanoes.Type[i] == "Stratovolcano":
        type_color = "green"
    elif volcanoes.Type[i] == "Complex volcano":
        type_color = "blue"
    elif volcanoes.Type[i] == "Shield volcano":
        type_color = "orange"
    elif volcanoes.Type[i] == "Lava dome":
        type_color = "pink"
    else:
        type_color = "purple"


    # Place the markers with the popup labels and data
    mymap.add_child(folium.Marker(location = coordinates,
                            popup =
                            "Year: " + str(volcanoes.Year[i]) + '<br>' +
                            "Name: " + str(volcanoes.Name[i]) + '<br>' +
                            "Country: " + str(volcanoes.Country[i]) + '<br>'
                            "Type: " + str(volcanoes.Type[i]) + '<br>'
                            "Coordinates: " + str(geo_df_list[i]),
                            icon = folium.Icon(color = "%s" % type_color)))
    i = i + 1
    
mymap

## **Self Help - You don't need to remember all of this!**

Honestly, you don't need to remember most of it. Here are the resources I use when looking for answers:

ChatGPT
* ChatGPT has quickly made huge changes to the programming landscape. It is a hugely powerful tool **If you use it the right way!**. I think it is a somewhat slippery slope of how to advise new programmers to use ChatGPT (or other AI tools) so I will refer to some best practices. My personal opinion is that you should use AI minimally when you are starting. When you have a better grasp of basic fundamentals, then you can include AI and greatly increase your speed. **Never accept ChatGPT code verbatim!** Always double check it before including it in your workflows.
* [How to Effectively Learn to Program w/ ChatGPT](https://towardsdatascience.com/how-to-effectively-start-coding-in-the-era-of-chatgpt-cfc5151e1c42)
* [Corey Schafer's "How to use ChatGPT"](https://www.youtube.com/watch?v=jRAAaDll34Q)

[Matplotlib Documentation](https://matplotlib.org/3.1.1/index.html)

[Stack Overflow](https://stackoverflow.com/) is a huge user community Q&A type site. Odds are very high that someone has 
asked your question before, just google something like "how to make scatter plot matlplotlib python". I'm pretty certain a 
StackOverflow thread will be one of the first few search results

*Stack Overflow Etiquette*
Don't just ask questions right away. Odds are high that for widely used packages, like matplotlib, a question and answer 
already exists. It is good practice to use that (and upvote it) if you like the answer. 

If you do ask a question, make sure it is specific and reproducible. People will downvote you and moderators will close the 
question if it is vague, incoherent, not-reproducible, or not clear in some other way. StackOverflow's purpose is to act as 
a reference guide, not as a forum to debate open ended questions such as "what is better, matplotlib or ggplot?". Go on 
Reddit if you want to do that. 